In [5]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
with open('intents.json') as file:
    data=json.load(file)

In [7]:
for intent in data['intents']:
    print(intent)

{'tag': 'greeting', 'patterns': ['hi', 'hey', 'is anyone there?', 'hello', 'heya', 'salute', 'hey jimmy', 'jimmy'], 'responses': ['Hello', 'Hi', 'Hi there', 'Heya']}
{'tag': 'goodbye', 'patterns': ['bye', 'goodbye', 'see you later', 'have a nice day', 'nice to meet you'], 'responses': ['Bye! Nice to meet you', 'Have a nice day', 'Goodbye', 'see You later']}
{'tag': 'thanks', 'patterns': ['thanks', 'thank you', "that's helpful", 'thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}
{'tag': 'about', 'patterns': ['who are you?', 'what are you?', 'who you are?'], 'responses': ['I.m Jimmy, your bot assistant', "I'm Jimmy, an Artificial Intelligent bot"]}
{'tag': 'name', 'patterns': ['what is your name', 'what should i call you', 'whats your name?'], 'responses': ['You can call me Jimmy.', "I'm Jimmy!", 'Just call me as Jimmy']}
{'tag': 'help', 'patterns': ['help', 'need help', 'assistance', 'can you help', 'can you help me', 'I need som

In [8]:
training_patterns=[]
training_label=[]
labels=[]
responses=[]

In [9]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_patterns.append(pattern)
        training_label.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [10]:
training_patterns

['hi',
 'hey',
 'is anyone there?',
 'hello',
 'heya',
 'salute',
 'hey jimmy',
 'jimmy',
 'bye',
 'goodbye',
 'see you later',
 'have a nice day',
 'nice to meet you',
 'thanks',
 'thank you',
 "that's helpful",
 'thanks for the help',
 'who are you?',
 'what are you?',
 'who you are?',
 'what is your name',
 'what should i call you',
 'whats your name?',
 'help',
 'need help',
 'assistance',
 'can you help',
 'can you help me',
 'I need some help']

In [11]:
training_label

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help']

In [12]:
labels

['greeting', 'goodbye', 'thanks', 'about', 'name', 'help']

In [13]:
responses

[['Hello', 'Hi', 'Hi there', 'Heya'],
 ['Bye! Nice to meet you', 'Have a nice day', 'Goodbye', 'see You later'],
 ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"],
 ['I.m Jimmy, your bot assistant', "I'm Jimmy, an Artificial Intelligent bot"],
 ['You can call me Jimmy.', "I'm Jimmy!", 'Just call me as Jimmy'],
 ['Please enter symptoms', 'What are your problems?', 'Enter Symptoms']]

In [14]:
num_classes=len(labels)

In [15]:
lbl_encoder=LabelEncoder()
lbl_encoder.fit(training_label)
training_label=lbl_encoder.transform(training_label)

In [16]:
training_label

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 5, 5, 5, 5, 0, 0, 0, 4, 4,
       4, 3, 3, 3, 3, 3, 3])

In [17]:
vocab_size=1000
max_len=20
oov_token="<OOV>"

tokenizer=Tokenizer(num_words=vocab_size,lower=True,oov_token=oov_token)
tokenizer.fit_on_texts(training_patterns)
#print(training_patterns)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_patterns)
#print(sequences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [18]:
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 17],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  7, 18, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 20],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 21],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 22],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  6,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 23],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 24],
       [ 0,  0,  0,  0,  0,  0

In [19]:
embedding_dim=16
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
epochs=1000
history=model.fit(padded_sequences,np.array(training_label),epochs=epochs)

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 1.7929 - accuracy: 0.1724
Epoch 2/1000
1/1 [==============================] - 0s 8ms/step - loss: 1.7912 - accuracy: 0.1724
Epoch 3/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.7897 - accuracy: 0.1379
Epoch 4/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.7887 - accuracy: 0.1379
Epoch 5/1000
1/1 [==============================] - 0s 21ms/step - loss: 1.7878 - accuracy: 0.1379
Epoch 6/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.7867 - accuracy: 0.1724
Epoch 7/1000
1/1 [==============================] - 0s 15ms/step - loss: 1.7857 - accuracy: 0.1724
Epoch 8/1000
1/1 [==============================] - 0s 17ms/step - loss: 1.7844 - accuracy: 0.1724
Epoch 9/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.7834 - accuracy: 0.1724
Epoch 10/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.7825 - accuracy: 0.1724
Epoch 11/100

1/1 [==============================] - 0s 14ms/step - loss: 1.6786 - accuracy: 0.4828
Epoch 84/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.6766 - accuracy: 0.4828
Epoch 85/1000
1/1 [==============================] - 0s 9ms/step - loss: 1.6746 - accuracy: 0.4828
Epoch 86/1000
1/1 [==============================] - 0s 8ms/step - loss: 1.6725 - accuracy: 0.4828
Epoch 87/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.6704 - accuracy: 0.4828
Epoch 88/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.6683 - accuracy: 0.4828
Epoch 89/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.6661 - accuracy: 0.4828
Epoch 90/1000
1/1 [==============================] - 0s 9ms/step - loss: 1.6639 - accuracy: 0.4828
Epoch 91/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.6616 - accuracy: 0.4828
Epoch 92/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.6593 - accuracy: 0.4828
Epoch 93/1000
1/1 

1/1 [==============================] - 0s 24ms/step - loss: 1.3446 - accuracy: 0.5172
Epoch 165/1000
1/1 [==============================] - 0s 21ms/step - loss: 1.3383 - accuracy: 0.5172
Epoch 166/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.3321 - accuracy: 0.5172
Epoch 167/1000
1/1 [==============================] - 0s 20ms/step - loss: 1.3257 - accuracy: 0.5172
Epoch 168/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.3196 - accuracy: 0.5172
Epoch 169/1000
1/1 [==============================] - 0s 16ms/step - loss: 1.3132 - accuracy: 0.5517
Epoch 170/1000
1/1 [==============================] - 0s 16ms/step - loss: 1.3072 - accuracy: 0.5517
Epoch 171/1000
1/1 [==============================] - 0s 16ms/step - loss: 1.3012 - accuracy: 0.5517
Epoch 172/1000
1/1 [==============================] - 0s 38ms/step - loss: 1.2951 - accuracy: 0.5517
Epoch 173/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.2890 - accuracy: 0.5517
Epoch

1/1 [==============================] - 0s 13ms/step - loss: 0.8306 - accuracy: 0.7586
Epoch 246/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.8235 - accuracy: 0.7586
Epoch 247/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.8166 - accuracy: 0.7586
Epoch 248/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.8098 - accuracy: 0.7586
Epoch 249/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.8029 - accuracy: 0.7586
Epoch 250/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.7959 - accuracy: 0.7586
Epoch 251/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7891 - accuracy: 0.7586
Epoch 252/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.7824 - accuracy: 0.7586
Epoch 253/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.7756 - accuracy: 0.7586
Epoch 254/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.7687 - accuracy: 0.7586
Epoch 

1/1 [==============================] - 0s 13ms/step - loss: 0.3349 - accuracy: 0.8966
Epoch 327/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.3306 - accuracy: 0.8966
Epoch 328/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.3263 - accuracy: 0.8966
Epoch 329/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.3221 - accuracy: 0.8966
Epoch 330/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.3179 - accuracy: 0.8966
Epoch 331/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.3139 - accuracy: 0.9310
Epoch 332/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.3097 - accuracy: 0.9310
Epoch 333/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.3060 - accuracy: 0.8966
Epoch 334/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.3021 - accuracy: 0.9310
Epoch 335/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.2980 - accuracy: 0.9310
Epoch 

...

1/1 [==============================] - 0s 10ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 651/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 652/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 653/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 654/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 655/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 656/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 657/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 658/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 659/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 14ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 732/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 733/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 734/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 735/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 736/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 737/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 738/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 739/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 740/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 9ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 813/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 814/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 815/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 816/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 817/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 818/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 819/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 820/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 821/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 14ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 894/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 895/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 896/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 897/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 898/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 899/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 900/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 901/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 902/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 10ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 975/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 976/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 977/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 978/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 979/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 980/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 981/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 982/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 983/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 

In [21]:
model.save("chatbot_model")
import pickle

with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)
    
with open ('label_encoder.pickle','wb') as ecn_file:
    pickle.dump(lbl_encoder,ecn_file,protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chatbot_model\assets
